In [3]:
import sys
from bs4 import BeautifulSoup  #网页解析，获取数据
import re                      #正则表达式，文字匹配
import urllib.request,urllib.error# 制定URL，获取数据
import xlwt                       #进行数据库操作
import sqlite3                    #s数据库操作
import urllib.request
import urllib.parse
import pandas as pd

In [14]:
findtype=re.compile(r'<p class="cx-name text-hover">(.*?)</p>')      #创建正则表达式，找名字
findprice=re.compile(r'<p class="cx-price">(.*?)</p>')     #找价格
findimg=re.compile(r'<img .*src="(.*?)"',re.S)     #找图片，换行符re.S图片
def main():
    askurl("http://car.bitauto.com/xuanchegongju/?l=8&mid=8")
    list=getdata("http://car.bitauto.com/xuanchegongju/?l=8&mid=8")
    name=['type','price','img']
    test=pd.DataFrame(columns=name,data=list)
    test.to_csv('result.csv',encoding='utf-8')
if __name__=="__main__": 
    main()


In [5]:
#浏览一个网页
def askurl(url):
    head={                    #模拟头部发送消息
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36"
    }#用户代理表示告诉网页我们伪装成浏览器，本质上告诉浏览器我们可以接受什么水平的文件,如果是418错误，一点不能错，大小写空格等
    request=urllib.request.Request(url,headers=head)
    html=""
    response=urllib.request.urlopen(request)
    html=response.read().decode("utf-8")
    #print(html)
    return html

In [7]:
#爬网页上的数据
def getdata(url):
    datalist=[]
    html=askurl(url)
    #解析数据
    soup =BeautifulSoup(html,"lxml")
    for items in soup.find_all('div',class_="search-result-list-item"):#查找符合要求的字符串,class要加下划线，因为是属性值
        #print(items)查看电影item
        data=[]
        items=str(items)
        type =re.findall(findtype,items)[0]
        #print(type)
        #re库查找指定的字符串
        data.append(type)
        price=re.findall(findprice,items)[0]
        data.append(price)
        #print(price)
        img='http:'+re.findall(findimg,items)[0]
        #print(img)
        data.append(img)
        datalist.append(data)
    #print(datalist)
    return datalist

In [16]:
import pandas as pd
tr1=pd.read_csv("result.csv",encoding='utf-8',index_col=0)
#tr1.drop([0,0],axis=1)
tr1
tr2=tr1['price'].str.extract("(.*?)万").iloc[:,0].str.split('-',expand=True).iloc[:,0]
tr3=tr1['price'].str.extract("(.*?)万").iloc[:,0].str.split('-',expand=True).iloc[:,1]
tr1.insert(1,"最低值",tr2)
tr1.insert(2,"最高值",tr3)
result=tr1.drop("price",axis=1)
result
result.to_csv('D:\\data analayse\\python\\last testing\\ProjectA\\result.csv',encoding='gbk')



